In [9]:
import psycopg2
conn = psycopg2.connect("""dbname='iot' user='gpadmin' host='10.110.122.107' port='5432' password='changeme'""")

In [10]:
import pandas.io.sql as psql
test_conn = psql.read_sql('select x, x+random() as y from generate_series(1,10) x;',conn)
print test_conn

    x          y
0   1   1.852321
1   2   2.987978
2   3   3.733503
3   4   4.905154
4   5   5.146880
5   6   6.016494
6   7   7.978725
7   8   8.040061
8   9   9.683253
9  10  10.275583


In [12]:
import plot from matplotlib as plt
plt.plot(1,2)

SyntaxError: invalid syntax (<ipython-input-12-2ff4559614f8>, line 1)

In [3]:
run sql/queries.py

In [113]:
input_schema = "iot_app"
input_table = "drilling_data_1000_arr"
output_schema = "iot_app"
output_table = "drilling_data_1000_arr_1hr_ahead"
sql = add_label_to_features(input_schema, input_table, output_schema, output_table)
print sql
df = psql.execute(sql, conn)
conn.commit()


        -- Flagging failure ONLY 1 hour ahead NOT for the whole run
        drop table if exists iot_app.drilling_data_1000_arr_1hr_ahead;
        create table iot_app.drilling_data_1000_arr_1hr_ahead 
        as 
        (
            select 
                case when failure_flag_for_full_run = 1 and ts_utc >= max_ts_utc_per_run - '1 hour'::interval then 1
                     else 0
                end::integer as flag_dep_var,
                random() as seed,
                *,
                rpm_stats_arr || rop_stats_arr || wob_stats_arr || flow_stats_arr || bitpos_stats_arr as indep_var_col,
                rpm_stats_arr_names || rop_stats_arr_names || wob_stats_arr_names || flow_stats_arr_names || bitpos_stats_arr_names as indep_var_col_names
            from (
                select 
                    *, 
                    max(ts_utc) over (partition by well_id, run_id) as max_ts_utc_per_run
                from iot_app.drilling_data_1000_arr
            ) t
        ) d

In [6]:
input_schema = "iot_app"
input_table = "drilling_data_1000_arr_1hr_ahead"
output_schema = "iot_app"
output_table = "drilling_data_1000_arr_1hr_ahead_tt"
sql, output_table_train, output_table_test, output_table_tt = create_train_and_test_set(input_schema, input_table, output_schema, output_table)
print sql, output_table_train, output_table_test, output_table_tt
df02 = psql.execute(sql, conn)
conn.commit()
sql = "select count(*) from iot_app.drilling_data_1000_arr_1hr_ahead_tt_train;"
df03 = psql.read_sql(sql, conn)
print df03
sql = "select count(*) from iot_app.drilling_data_1000_arr_1hr_ahead_tt_test;"
df04 = psql.read_sql(sql, conn)
print df04
sql = "select count(*) from iot_app.drilling_data_1000_arr_1hr_ahead_tt_traintest;"
df04b = psql.read_sql(sql, conn)
print df04b

  
        -- Training table
        drop table if exists iot_app.drilling_data_1000_arr_1hr_ahead_tt_train;
        create table iot_app.drilling_data_1000_arr_1hr_ahead_tt_train
        as
        (
            select 
                *
            from 
                iot_app.drilling_data_1000_arr_1hr_ahead
            where 
                seed <= 0.70
                and
                -- Removing wells with NULL values in the features
                (well_id, run_id, window_id, global_window_id) not in (
                    select well_id, run_id, window_id, global_window_id
                    from ( 
                        select well_id, run_id, window_id, global_window_id, flag_dep_var, seed, unnest(indep_var_col) as uval
                        from iot_app.drilling_data_1000_arr_1hr_ahead
                    ) t1
                    where uval is NULL
                    group by 1,2,3,4
                    )
        ) distributed by (global_window_id);
        
     

In [117]:
input_schema = "iot_app"
input_table = "drilling_data_1000_arr_1hr_ahead_tt_train"
output_schema = "iot_app"
output_table = "drilling_data_1000_arr_1hr_ahead_tt_train_modelcoef"
sql = train_model(input_schema, input_table, output_schema, output_table)
print sql
df05 = psql.execute(sql, conn)
conn.commit()
sql = "select count(*) from iot_app.drilling_data_1000_arr_1hr_ahead_tt_train_modelcoef;"
df = psql.read_sql(sql, conn)
print df


        drop table if exists iot_app.drilling_data_1000_arr_1hr_ahead_tt_train_modelcoef;
        select madlib.elastic_net_train(
            'iot_app.drilling_data_1000_arr_1hr_ahead_tt_train',
            'iot_app.drilling_data_1000_arr_1hr_ahead_tt_train_modelcoef',
            'flag_dep_var',
            'indep_var_col',
            'binomial',
            1.0,
            0.001
        );    
    
   count
0      1


In [7]:
mdl_schema = "iot_app"
mdl_table = "drilling_data_1000_arr_1hr_ahead_tt_train_modelcoef"
scoring_schema = "iot_app"
scoring_table = "drilling_data_1000_arr_1hr_ahead_tt_traintest"
output_schema = "iot_app"
output_table = "drilling_data_1000_arr_1hr_ahead_tt_traintest_scored"
sql = predict_model(mdl_schema, mdl_table, scoring_schema, scoring_table, output_schema, output_table)
print sql
df07 = psql.execute(sql, conn)
conn.commit()


        drop table if exists iot_app.drilling_data_1000_arr_1hr_ahead_tt_traintest_scored;
        create table iot_app.drilling_data_1000_arr_1hr_ahead_tt_traintest_scored
        as
        (
            select 
                   *,
                   madlib.elastic_net_binomial_predict(coef_all, intercept, indep_var_col) as pred,
                   madlib.elastic_net_binomial_prob (coef_all, intercept, indep_var_col) as prob
            from iot_app.drilling_data_1000_arr_1hr_ahead_tt_train_modelcoef mdl,
                 iot_app.drilling_data_1000_arr_1hr_ahead_tt_traintest score
        ) distributed randomly;
    


In [121]:
sql = "select count(*) from iot_app.drilling_data_1000_arr_1hr_ahead_tt_traintest_scored;"
df08 = psql.read_sql(sql, conn)
print df08

     count
0  8662960


In [103]:
conn.close()